# **Day14**: *Parabolic Reflector Dish*

## Part 1

You reach the place where all of the mirrors were pointing: a massive parabolic reflector dish attached to the side of another large mountain.

The dish is made up of many small mirrors, but while the mirrors themselves are roughly in the shape of a parabolic reflector dish, each individual mirror seems to be pointing in slightly the wrong direction. If the dish is meant to focus light, all it's doing right now is sending it in a vague direction.

This system must be what provides the energy for the lava! If you focus the reflector dish, maybe you can go where it's pointing and use the light to fix the lava production.

Upon closer inspection, the individual mirrors each appear to be connected via an elaborate system of ropes and pulleys to a large metal platform below the dish. The platform is covered in large rocks of various shapes. Depending on their position, the weight of the rocks deforms the platform, and the shape of the platform controls which ropes move and ultimately the focus of the dish.

In short: if you move the rocks, you can focus the dish. The platform even has a control panel on the side that lets you tilt it in one of four directions! The rounded rocks (O) will roll when the platform is tilted, while the cube-shaped rocks (#) will stay in place. You note the positions of all of the empty spaces (.) and rocks (your puzzle input). For example:

```
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
```

Start by tilting the lever so all of the rocks will slide north as far as they will go:

```
OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....
```

You notice that the support beams along the north side of the platform are damaged; to ensure the platform doesn't collapse, you should calculate the total load on the north support beams.

The amount of load caused by a single rounded rock (O) is equal to the number of rows from the rock to the south edge of the platform, including the row the rock is on. (Cube-shaped rocks (#) don't contribute to load.) So, the amount of load caused by each rock in each row is as follows:

```
OOOO.#.O.. 10
OO..#....#  9
OO..O##..O  8
O..#.OO...  7
........#.  6
..#....#.#  5
..O..#.O.O  4
..O.......  3
#....###..  2
#....#....  1
```

The total load is the sum of the load caused by all of the rounded rocks. In this example, the total load is 136.

Tilt the platform so that the rounded rocks all roll north. Afterward, what is the total load on the north support beams?

### Solution

In [1]:
def move_rock(row, col, lines):
    new_row, new_col = row, col
    while new_row > 0 and lines[new_row-1][new_col] == '.':
        new_row -= 1
    
    lines[row][col] = '.'
    lines[new_row][new_col] = 'O'
    return lines
            

In [2]:
def calculate_load(input_string):
    lines = [list(line) for line in input_string.splitlines()]
    l = len(lines)
    
    for row in range(1, len(lines)):
        for col in range(len(lines[0])):
            if lines[row][col] == 'O':
                lines = move_rock(row, col, lines)

    load = []
    for row, line in enumerate(lines):
        for col, item in enumerate(line):
            if item == 'O':
                load.append(l-row)

    return sum(load)


### Example

In [3]:
input_string = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""


calculate_load(input_string)


136

### Submission

In [4]:
with open('input/day14.txt', 'r') as file:
    input_string = file.read()

calculate_load(input_string)


110090

## Part 2

The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "spin cycle" attempts to do just that!

Each cycle tilts the platform four times so that the rounded rocks roll north, then west, then south, then east. After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.

Here's what happens in the example above after each of the first few cycles:

After 1 cycle:


```
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....
```

After 2 cycles:

```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O
```

After 3 cycles:
```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
```

This process should work if you leave it running long enough, but you're still worried about the north support beams. To make sure they'll survive for a while, you need to calculate the total load on the north support beams after 1000000000 cycles.

In the above example, after 1000000000 cycles, the total load on the north support beams is 64.

Run the spin cycle for 1000000000 cycles. Afterward, what is the total load on the north support beams?

### Solution

In [5]:
def flip(lines):
    return lines[::-1]

def transpose(lines):
    return list(map(list, zip(*lines))) 

direction_transform = {
    'N': lambda lines: lines,
    'E': lambda lines: flip(transpose(lines)),
    'S': lambda lines: flip(lines),
    'W': lambda lines: transpose(lines),
}

direction_revert = {
    'N': lambda lines: lines,
    'E': lambda lines: transpose(flip(lines)),
    'S': lambda lines: flip(lines),
    'W': lambda lines: transpose(lines),
}

def move_rock(position, lines):
    row, col = position
    while row > 0 and lines[row-1][col] == '.':
        row -= 1

    old_row, old_col = position
    lines[old_row][old_col] = '.'
    lines[row][col] = 'O'
    return lines


def tilt_platform(direction, lines):
    # transform lines
    lines = direction_transform[direction](lines)
    
    # move rocks to top of `lines`
    for row, line in enumerate(lines):
        for col, item in enumerate(line):
            if item == "O":
                lines = move_rock((row, col), lines)
    
    # revert lines transformation
    lines = direction_revert[direction](lines)
    return lines

def run_cycle(lines):
    lines = tilt_platform('N', lines)
    lines = tilt_platform('W', lines)
    lines = tilt_platform('S', lines)
    lines = tilt_platform('E', lines)
    return lines

In [6]:
from copy import deepcopy


def calculate_load(input_string):
    lines = [list(line) for line in input_string.splitlines()]
    l = len(lines)

    history = [lines]
    while True:
        lines = run_cycle(lines)
        if lines in history: 
            index = history.index(lines)
            cycle_length = len(history)-index
            break
        history.append(deepcopy(lines))
    
    
    total_cycles = 1_000_000_000
    offset = index - 1
    remaining_cycles = ((total_cycles - offset) % cycle_length) - 1
    lines = history[index+remaining_cycles]


    load = []
    for row, line in enumerate(lines):
        for col, item in enumerate(line):
            if item == 'O':
                load.append(l-row)

    return sum(load)


### Example

In [7]:
input_string = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""


calculate_load(input_string)


64

### Submission

In [8]:
with open('input/day14.txt', 'r') as file:
    input_string = file.read()

calculate_load(input_string)


95254